# Exploring Seattle Airbnb Data
This is the first project for Udacity's Data Scientist Nanodegree. The data used in this project was obtained from Kaggle "Seattle Airbnb Open Data" The whole project is implemented following "Cross-Industry Standard Process for Data Mining (CRISP-DM)" process, an industry standard for data analysis. It includes 6 steps: 

1. Business Understanding.
2. Data Understanding.
3. Prepare Data.
4. Data Modeling.
5. Evaluate the Results
6. Deploy.

In this project, we will follow these steps to dive into the data. 




## 1. Business Understanding
The three questions that I will answer by exploiring the data are: 
1. What is the rank of hosts with higest review scores?
2. What aspects correlate well to review scores?
3. 

# 2. Data Understanding


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

calendar = pd.read_csv("./data/seattle/calendar.csv")
listings = pd.read_csv("./data/seattle/listings.csv")
reviews = pd.read_csv("./data/seattle/reviews.csv")
sof = pd.read_csv("../developer_survey_2019/survey_results_public.csv")
calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [3]:
sof.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,30.0,Man,No,Straight / Heterosexual,White or of European descent;Multiracial,No,Appropriate in length,Easy


In [6]:
sof.dtypes

Respondent          int64
MainBranch         object
Hobbyist           object
OpenSourcer        object
OpenSource         object
Employment         object
Country            object
Student            object
EdLevel            object
UndergradMajor     object
EduOther           object
OrgSize            object
DevType            object
YearsCode          object
Age1stCode         object
YearsCodePro       object
CareerSat          object
JobSat             object
MgrIdiot           object
MgrMoney           object
MgrWant            object
JobSeek            object
LastHireDate       object
LastInt            object
FizzBuzz           object
JobFactors         object
ResumeUpdate       object
CurrencySymbol     object
CurrencyDesc       object
CompTotal         float64
                   ...   
Containers         object
BlockchainOrg      object
BlockchainIs       object
BetterLife         object
ITperson           object
OffOn              object
SocialMedia        object
Extraversion

In [7]:
sof['CompTotal']

0             NaN
1             NaN
2         23000.0
3         61000.0
4             NaN
5         40000.0
6             NaN
7             NaN
8        138000.0
9        950000.0
10            NaN
11            NaN
12        90000.0
13         4150.0
14            NaN
15        29000.0
16        90000.0
17       120000.0
18         9500.0
19         3000.0
20            NaN
21       103000.0
22        69000.0
23         8000.0
24         7000.0
25       114000.0
26            NaN
27            NaN
28         2000.0
29        90000.0
           ...   
88853         NaN
88854         NaN
88855         NaN
88856         NaN
88857         NaN
88858         NaN
88859         NaN
88860         NaN
88861         NaN
88862         NaN
88863         NaN
88864         NaN
88865         NaN
88866         NaN
88867         NaN
88868         NaN
88869         NaN
88870         NaN
88871         NaN
88872         NaN
88873         NaN
88874         NaN
88875         NaN
88876         NaN
88877     

In [8]:
num_df = sof.select_dtypes(include=['float64']).copy()
num_df.head()

,CompTotal,ConvertedComp,WorkWeekHrs,CodeRevHrs,Age
0,NaN,NaN,NaN,NaN,14.0
1,NaN,NaN,NaN,NaN,19.0
2,23000.0,8820.0,40.0,NaN,28.0
3,61000.0,61000.0,80.0,NaN,22.0
4,NaN,NaN,55.0,NaN,30.0


In [10]:
women = sof[sof["Gender"]== "Woman"].head()
women 

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
11,12,I am a student who is learning to code,No,Never,"OSS is, on average, of HIGHER quality than pro...",Employed part-time,Canada,"Yes, full-time",Some college/university study without earning ...,Mathematics or statistics,...,Just as welcome now as I felt last year,Courses on technologies you're interested in,21.0,Woman,No,Straight / Heterosexual,Black or of African descent,No,Appropriate in length,Easy
30,31,I am a student who is learning to code,No,Never,NaN,"Not employed, and not looking for work",Canada,No,Primary/elementary school,NaN,...,NaN,NaN,NaN,Woman,No,NaN,East Asian,No,Too long,Neither easy nor difficult
34,35,I am a developer by profession,No,Never,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Information systems, information technology, o...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,35.0,Woman,No,Straight / Heterosexual,Black or of African descent,No,Appropriate in length,Easy
50,51,I am a student who is learning to code,Yes,Never,"OSS is, on average, of HIGHER quality than pro...",Employed part-time,United States,"Yes, full-time",Some college/university study without earning ...,"Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers,18.0,Woman,No,Straight / Heterosexual,South Asian,No,Too long,Easy
70,71,I am a developer by profession,Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Philippines,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,A lot more welcome now than last year,Tech articles written by other developers;Tech...,21.0,Woman,No,Straight / Heterosexual,East Asian;South Asian,No,Appropriate in length,Easy


In [11]:
women_salary = women.select_dtypes(include=['float64']).copy()
women_salary

,CompTotal,ConvertedComp,WorkWeekHrs,CodeRevHrs,Age
11,NaN,NaN,NaN,NaN,21.0
30,NaN,NaN,NaN,NaN,NaN
34,80000.0,80000.0,40.0,0.5,35.0
50,NaN,NaN,NaN,NaN,18.0
70,18000.0,4128.0,8.0,NaN,21.0


In [17]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [1]:
%pwd

'/Users/jaywang/Documents/online_courses/udacity_data_science/airbnb_project'

In [3]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


3818

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

# 3. Prepare Data

# 4. Model Data

# 5. Results

# 6. Deploy